# Dataset and Evaluation Test

In [1]:
%matplotlib inline
import sys
sys.path.append("../")

import cv2
from tqdm import tqdm
import matplotlib as mpl
import matplotlib.pyplot as plt
import pylab
import numpy as np
import scipy.io as scio

import torch
from PIL import Image

import quat_math as qm
from generic_pose.utils import to_np, to_var


import warnings; warnings.simplefilter('ignore')
pylab.rcParams['figure.figsize'] = 20, 12

## Useful display functions

In [2]:
def imshowCV(img, axis = False, show = True):
    if not axis:
        plt.axis('off')
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if(show):
        plt.show()
    
def imshow(img, axis = False, colorbar = False, show = True):
    if not axis:
        plt.axis('off')
    plt.imshow(img)
    if(colorbar):
        plt.colorbar()
    if(show):
        plt.show()
    
def torch2Img(img, show = True):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    disp_img = to_np(img)
    if len(disp_img.shape) == 4:
        disp_img = disp_img[0]
    disp_img = disp_img.transpose((1,2,0))
    disp_img = disp_img * std + mean
    return disp_img
    
#from generic_pose.utils.image_preprocessing import unprocessImages
def imshowTorch(img, axis = False, show = True):
    if not axis:
        plt.axis('off')
    disp_img = torch2Img(img)
    plt.imshow(disp_img.astype(np.uint8))
    #plt.imshow(unprocessImages(img)[0])
    if(show):
        plt.show()

def plotImageScatter(img, choose, show = True):
    coords = np.unravel_index(choose, img.shape[:2])    
    plt.axis('off')
    plt.imshow(img.astype(np.uint8))    
    plt.scatter(coords[1], coords[0], 50)
    #plt.colorbar()
    if(show):
        plt.show()

## Location of YCB Dataset

In [1]:
dataset_root = '../datasets/ycb/YCB_Video_Dataset'


## Location of Dense Fusion Weights File

In [4]:
model_checkpoint = '/path/to/model/weights.pth'

## Inialize Estimator

In [5]:
from dense_fusion.evaluate import DenseFusionEstimator, preprocessData
from dense_fusion.data_processing import preprocessPoseCNNMetaData

num_points = 1000
num_obj = 21
df_estimator = DenseFusionEstimator(num_points, num_obj, model_checkpoint)

## New Dataloader

In [6]:
from object_pose_utils.datasets.pose_dataset import OutputTypes as otypes

output_format = [otypes.OBJECT_LABEL,
                 otypes.QUATERNION,
                 otypes.IMAGE_CROPPED,
                 otypes.DEPTH_POINTS_MASKED_AND_INDEXES]

object_list = [14]
mode = "train_syn"

from object_pose_utils.datasets.ycb_dataset import YcbDataset as YCBDataset
from object_pose_utils.datasets.ycb_dataset import YcbImagePreprocessor

dataset = YCBDataset(dataset_root, mode=mode, 
                     object_list = object_list, 
                     output_data = output_format, 
                     preprocessor = YcbImagePreprocessor,
                     image_size = [640, 480], num_points=1000)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

In [7]:
idx = 0
#data = dataset.__getitem__(idx)
for data in dataloader:
    break
obj, q, im, pts, chs = data

pred_r, pred_t, pred_c = df_estimator.estimatePose(im, pts, chs, obj-1, return_all=False)
pred_r = pred_r[[1,2,3,0]]
print(obj)
print(pred_r)
print(q)
print(pred_t)

#imshowTorch(im)

tensor([[14]])
tensor([ 0.7652, -0.4055,  0.2423,  0.4375], grad_fn=<IndexBackward>)
tensor([[ 0.7850, -0.3913,  0.2590,  0.4044]])
tensor([ 0.0571, -0.0190,  0.6155], grad_fn=<ViewBackward>)


## Old Data Loading

In [8]:
from dense_fusion.evaluate_likelihood import getYCBClassData, getYCBGroundtruth

test_filenames = '{0}/image_sets/train_split.txt'.format(dataset_root)
with open(test_filenames) as f:
    file_list = f.read().split()

data_prefix = '{0}/data/{1}'.format(dataset_root, file_list[0])

img = Image.open('{}-color.png'.format(data_prefix))
depth = np.array(Image.open('{}-depth.png'.format(data_prefix)))
pose_meta = scio.loadmat('{}-meta.mat'.format(data_prefix))
posecnn_meta = scio.loadmat('{}-posecnn.mat'.format(data_prefix))
object_classes = set(pose_meta['cls_indexes'].flat) & \
                    set(posecnn_meta['rois'][:,1:2].flatten().astype(int))
cls_idx = 14 #list(object_classes)[0]
obj_idx = np.nonzero(posecnn_meta['rois'][:,1].astype(int) == cls_idx)[0][0]
mask, bbox, object_label = preprocessPoseCNNMetaData(posecnn_meta, obj_idx)

q_gt, _ = getYCBGroundtruth(pose_meta, posecnn_meta, obj_idx)
q_gt = torch.Tensor(q_gt).unsqueeze(0)
if(torch.cuda.is_available()):
    q_gt = q_gt.cuda()


pred_r, pred_t = df_estimator(img, depth, mask, bbox, object_label)
pred_r = pred_r[[1,2,3,0]]
print(object_label)
print(pred_r)
print(q_gt)
print(pred_t)
#imshow(img)

13.0
tensor([ 0.7738, -0.3887,  0.2321,  0.4431], grad_fn=<IndexBackward>)
tensor([[ 0.7850, -0.3913,  0.2590,  0.4044]])
tensor([ 0.0567, -0.0181,  0.6145], grad_fn=<ViewBackward>)
